In [1]:
import pandas as pd

# O arquivo deve estar na subpasta 'data/' do seu projeto.
FILE_PATH = 'data/adult.csv'

# 1. Carregar o dataset. O 'skipinitialspace=True' é essencial
# para limpar os espaços em branco que acompanham os valores string do dataset Adult.
try:
    df = pd.read_csv(FILE_PATH, na_values='?', skipinitialspace=True)
except FileNotFoundError:
    print(f"ERRO: Arquivo não encontrado em {FILE_PATH}. Verifique se o nome e o caminho estão corretos.")
    raise # Interrompe a execução se o arquivo não for encontrado

# 2. Limpeza e Tratamento de Dados
# Remove espaços extras em nomes de colunas e na coluna alvo (income)
df.columns = df.columns.str.strip() 
df['income'] = df['income'].str.strip() 
# Remove linhas com valores ausentes ('?') que foram mapeados para NaN
df = df.dropna() 

# --- Definição dos Atributos de Fairness (FAT) ---
ATRIBUTO_PROTEGIDO = 'sex'  # Gênero será o atributo que auditaremos
ALVO = 'income'             # O rótulo alvo (quem ganha >50K, proxy para aprovação de crédito)

# 3. Calcular a Taxa de Sucesso (>50K) para cada grupo de gênero
# normalize='index' calcula a proporção de sucesso DENTRO de cada grupo de 'sex'.
disparidade_por_sexo = pd.crosstab(df[ATRIBUTO_PROTEGIDO], df[ALVO], normalize='index') * 100

# 4. Extrair e Imprimir as Taxas de Sucesso
taxa_sucesso_por_sexo = disparidade_por_sexo['>50K']

taxa_f = taxa_sucesso_por_sexo['Female']
taxa_m = taxa_sucesso_por_sexo['Male']

# 5. Calcular o DISPARATE IMPACT (DI)
# DI = (Taxa de Sucesso do Grupo Não Privilegiado) / (Taxa de Sucesso do Grupo Privilegiado)
# Assume-se que 'Female' é o grupo não privilegiado neste contexto histórico/social de renda.
di = taxa_f / taxa_m

print("--- DIAGNÓSTICO DE VIÉS NO DATASET ---")
print(f"Taxa de Sucesso (>50K) Masculina: {taxa_m:.2f}%")
print(f"Taxa de Sucesso (>50K) Feminina:  {taxa_f:.2f}%")
print("---------------------------------------")
print(f"DISPARATE IMPACT (DI) Feminino/Masculino: {di:.2f}")

# Critério de 80% (A Regra de Ouro/Quatro Quintos)
if di < 0.8:
    print("\nAVISO CRÍTICO: O Viés de Dados está presente (DI < 0.8). O modelo herdará a discriminação.")
else:
    print("\nO Viés de Dados está dentro do limite aceitável de 80%.")

--- DIAGNÓSTICO DE VIÉS NO DATASET ---
Taxa de Sucesso (>50K) Masculina: 31.38%
Taxa de Sucesso (>50K) Feminina:  11.37%
---------------------------------------
DISPARATE IMPACT (DI) Feminino/Masculino: 0.36

AVISO CRÍTICO: O Viés de Dados está presente (DI < 0.8). O modelo herdará a discriminação.
